## Model hyperparametr tuning

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

#### Data preparation

In [2]:
data = pd.read_csv('data/telecom_churn.csv')
data.head(3)

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2


In [3]:
ct = ColumnTransformer([
    ('binary data', OneHotEncoder(), [2,3]),
    ('standard', StandardScaler(), [1,4,5,6,7,8,9,10])],
    remainder = 'passthrough')

data =  ct.fit_transform(data)

In [4]:
data[:2]

array([[ 0.        ,  1.        ,  0.        ,  1.        ,  0.67648946,
         1.48020373, -0.42793202,  1.56676695,  0.47664315,  1.99072703,
        -0.0715836 , -0.08500823,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  1.        ,  0.14906505,
         2.26607248, -0.42793202, -0.33373776,  1.12450284,  1.56451025,
        -0.10708191,  1.24048169,  0.        ]])

In [5]:
x_train, x_test, y_train, y_test = train_test_split(pd.DataFrame(data).iloc[:,1:].values, pd.DataFrame(data)[0], test_size=0.2, random_state=2077)

In [6]:
x_train.shape

(2666, 12)

### Neural network

In [7]:
param = {
    'hidden_layer_sizes' : [10,50,100,200,300,400,500],
    'activation' : ['identity', 'logistic', 'tanh', 'relu'],
    'alpha' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'batch_size ' : ['auto', 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'learning_rate' : ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'power_t' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'max_iter' : [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'shuffle' : [True, False],
    'momentum' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
}

In [8]:
HPT_NN = GridSearchCV(MLPClassifier(),param, n_jobs = 2, verbose = 1)
HPT_NN.fit(x_train, y_train)

In [ ]:
best = HPT_NN.best_estimator_
HPT_NN.best_estimator_

In [ ]:
s = pickle.dump(HPT_NN, open('models/NN_tuned.pkl', 'wb'))

## XGBoost

In [ ]:
param = {
    'n_estimators' : [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'max_depth' : [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'max_leaves ' : [0, 2, 3, 4, 5, 6]
}

In [ ]:
HPT_XGB = GridSearchCV(XGBClassifier(),param_distributions=param, n_jobs = 2, verbose = 1)
HPT_XGB.fit(x_train, y_train)

In [ ]:
best = HPT_XGB.best_estimator_
HPT_XGB.best_estimator_

In [ ]:
s = pickle.dump(HPT_XGB, open('models/XGB_tuned.pkl', 'wb'))

#### Dalex - model agnostic explainer

In [ ]:
import dalex as dx

## Ensemble model